# Positional Model Training - LSTM with MDS Embeddings

This notebook trains an LSTM model for badminton shot classification using positional features.
It uses Multi-Dimensional Scaling (MDS) to create embeddings from positional data and applies
hyperparameter tuning with Keras Tuner for optimal model performance.

## Data Exploration

First, let's explore the structure of our extracted features data.

In [ ]:
import os

# Analyze the number of files in each shot type subfolder
data_path = "../VB_DATA/extracted_features_positional_green_20_normalized/"
subfolders = [f.name for f in os.scandir(data_path) if f.is_dir()]
file_counts = {}

for folder in subfolders:
    file_counts[folder] = len(os.listdir(os.path.join(data_path, folder)))

print("File counts in each shot type subfolder:")
for subfolder, count in file_counts.items():
    print(f"{subfolder}: {count}")

## Data Loading and Preprocessing

Load the positional data and prepare it for training. We'll use a balanced sampling approach
to ensure equal representation across shot types.

In [ ]:
import numpy as np
import pandas as pd
import ast
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers, models
import tensorflow as tf

# Configuration
TARGET_FRAMES = 20
TARGET_SAMPLES = 138  # Number of samples per category for balanced dataset

# Selected shot categories based on data availability
CATEGORIES = [
    '00_Short_Serve', '02_Lift', '04_Block',
    '05_Drop_Shot', '06_Push_Shot', '08_Cut',
    '12_Clear', '13_Long_Serve', '14_Smash',
    '15_Flat_Shot'
]

def load_data(base_dir):
    """Load and balance the dataset across all shot categories."""
    X, y = [], []
    
    for idx, category in enumerate(CATEGORIES):
        cat_dir = os.path.join(base_dir, category)
        
        # Collect all valid sequences for this category
        frames = []
        ids = []
        
        for fname in os.listdir(cat_dir):
            if not fname.endswith('.csv'):
                continue
                
            df = pd.read_csv(os.path.join(cat_dir, fname))
            if 'Frame' in df.columns:
                df = df.drop(columns=['Frame'])
            
            sequence = df.values.tolist()
            
            # Only include sequences with sufficient frames
            if len(sequence) >= TARGET_FRAMES:
                frames.append(sequence)
                ids.append(idx)
        
        # Randomly sample TARGET_SAMPLES from available sequences
        if len(frames) >= TARGET_SAMPLES:
            random_indices = np.random.choice(len(frames), TARGET_SAMPLES, replace=False)
            for i in random_indices:
                X.append(frames[i])
                y.append(ids[i])
        else:
            print(f"Warning: {category} has only {len(frames)} sequences, less than target {TARGET_SAMPLES}")
            # Use all available sequences
            X.extend(frames)
            y.extend(ids)
    
    X = np.stack(X, axis=0)
    y = tf.keras.utils.to_categorical(y, num_classes=len(CATEGORIES))
    
    return X, y, CATEGORIES

## MDS Embedding Generation

Create embeddings using Multi-Dimensional Scaling (MDS) from synthetic positional data.
This transforms discrete positional codes into continuous vector representations.

In [ ]:
import os
import ast
import numpy as np
import pandas as pd
from itertools import product
from sklearn.manifold import MDS
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Masking
from tensorflow.keras.optimizers import Adam

# Load synthetic dataset for MDS embedding generation
df = pd.read_csv('../synthetic_data/synthetic_dataset_30deg.csv')
labels = df['point_name'].values
n = len(labels)

# Extract 2D and 3D coordinates from the dataset
coords2 = {row['point_name']: ast.literal_eval(row['point2']) for _, row in df.iterrows()}
coords3 = {row['point_name']: ast.literal_eval(row['point3']) for _, row in df.iterrows()}

def euclidean_distance(a, b):
    """Calculate Euclidean distance between two points."""
    return np.hypot(a[0] - b[0], a[1] - b[1])

# Compute distance matrices
n = len(labels)
D_geo = np.zeros((n, n))
D_sum = np.zeros((n, n))
D_mean = np.zeros((n, n))

for i, j in product(range(n), range(n)):
    lab_i, lab_j = labels[i], labels[j]
    d2 = euclidean_distance(coords2[lab_i], coords2[lab_j])
    d3 = euclidean_distance(coords3[lab_i], coords3[lab_j])

    D_geo[i, j] = np.sqrt(d2 * d3)  # Geometric mean
    D_sum[i, j] = d2 + d3           # Sum
    D_mean[i, j] = 0.5 * (d2 + d3)  # Arithmetic mean

# Generate embeddings using MDS
mds = MDS(n_components=4, dissimilarity='precomputed', random_state=42)
embeddings = mds.fit_transform(D_mean)

# Create lookup dictionary from positional code to embedding vector
code2emb = {lab: embeddings[i] for i, lab in enumerate(labels)}

print(f"Generated {len(code2emb)} positional embeddings with {embeddings.shape[1]} dimensions")

## Sequence Data Preparation

Transform the positional sequences into embedding sequences for LSTM training.

In [ ]:
# Build sequence dataset with embeddings
DATA_DIR = "../VB_DATA/extracted_features_positional_green_20_normalized"
CATEGORIES = [
    '00_Short_Serve', '02_Lift', '05_Drop_Shot', '08_Cut',
    '12_Clear', '13_Long_Serve', '14_Smash', '15_Flat_Shot'
]

all_seqs, all_labels = [], []

for folder in CATEGORIES:
    folder_path = f"{DATA_DIR}/{folder}"
    label = folder.split('_', 1)[1]  # Extract shot name from folder
    
    for fname in sorted(os.listdir(folder_path)):
        if not fname.endswith('.csv'):
            continue
            
        df = pd.read_csv(f"{folder_path}/{fname}")
        if 'Frame' in df:
            df = df.drop(columns=['Frame'])
        
        # Convert positional codes to embedding vectors
        seq_codes = df.values.astype(str)
        
        # Map each code to its embedding vector: (frames, points, emb_dim)
        seq_vecs = np.stack([
            np.stack([code2emb[code] for code in row], axis=0)
            for row in seq_codes
        ], axis=0)
        
        # Flatten points into features: (frames, points * emb_dim)
        T, P, E = seq_vecs.shape
        seq_flat = seq_vecs.reshape(T, P * E)
        
        all_seqs.append(seq_flat)
        all_labels.append(label)

# Pad or truncate sequences to fixed length
from tensorflow.keras.preprocessing.sequence import pad_sequences
X = pad_sequences(all_seqs, maxlen=20, dtype='float32', padding='post', truncating='post')

# Encode labels
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(all_labels)
y = np.eye(len(le.classes_))[y]  # One-hot encoding

print(f"Dataset shape: {X.shape}")
print(f"Labels shape: {y.shape}")
print(f"Shot classes: {le.classes_}")

## Hyperparameter Tuning with Keras Tuner

Use Keras Tuner to find optimal hyperparameters for the LSTM model.

In [ ]:
import kerastuner as kt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Masking, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Split data into train/validation sets
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# Further split for test set
X_train, X_test, y_train, y_test = train_test_split(
    X_train, y_train, test_size=0.2, stratify=y_train, random_state=42
)

def build_model(hp):
    """Build LSTM model with hyperparameter tuning."""
    model = Sequential()
    
    # Masking layer for padded sequences
    model.add(Masking(mask_value=0.0, input_shape=X_train.shape[1:]))
    
    # Tune number of LSTM layers and their units
    num_layers = hp.Int('num_lstm_layers', 1, 2)
    for i in range(num_layers):
        return_sequences = (i < num_layers - 1)
        units = hp.Int(f'lstm_units_{i}', min_value=32, max_value=256, step=32)
        model.add(LSTM(units, return_sequences=return_sequences))
        
        # Add dropout for regularization
        dropout_rate = hp.Float(f'dropout_{i}', 0.0, 0.5, step=0.1)
        model.add(Dropout(dropout_rate))
    
    # Output layer
    model.add(Dense(y_train.shape[1], activation='softmax'))
    
    # Tune learning rate
    learning_rate = hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])
    
    model.compile(
        optimizer=Adam(learning_rate=learning_rate),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# Initialize Keras Tuner
tuner = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=30,
    factor=3,
    directory='kt_dir',
    project_name='lstm_sequence_tuning'
)

# Early stopping callback
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

# Run hyperparameter search
print("Starting hyperparameter search...")
tuner.search(
    X_train, y_train,
    epochs=50,
    validation_data=(X_val, y_val),
    callbacks=[stop_early],
    batch_size=32
)

# Get best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print("\nBest hyperparameters:")
print(f"  LSTM layers: {best_hps.get('num_lstm_layers')}")
print(f"  Learning rate: {best_hps.get('learning_rate')}")

# Train final model with best hyperparameters
print("\nTraining final model...")
model = tuner.hypermodel.build(best_hps)
history = model.fit(
    X_train, y_train,
    epochs=50,
    validation_data=(X_val, y_val),
    batch_size=32,
    callbacks=[stop_early],
    verbose=1
)

# Evaluate on test set
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(f"\nTest accuracy: {test_acc:.4f}")

## Model Evaluation

Evaluate the trained model with detailed metrics including confusion matrix and class-wise accuracy.

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, top_k_accuracy_score
import pandas as pd

# Generate predictions
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

# Confusion matrix
cm = confusion_matrix(y_true_classes, y_pred_classes, labels=np.arange(len(le.classes_)))
cm_df = pd.DataFrame(cm, index=le.classes_, columns=le.classes_)

print("Confusion Matrix:")
print(cm_df)

# Class-wise accuracy
class_accuracies = cm.diagonal() / cm.sum(axis=1)
overall_accuracy = accuracy_score(y_true_classes, y_pred_classes)

print("\nClass Accuracies:")
for cls, acc in zip(le.classes_, class_accuracies):
    print(f"{cls}: {acc:.4f}")

print(f"\nOverall Accuracy: {overall_accuracy:.4f}")

# Top-k accuracy
top_3_acc = top_k_accuracy_score(y_true_classes, y_pred, k=3)
print(f"Top-3 Accuracy: {top_3_acc:.4f}")

## SHAP Analysis for Model Interpretability

Use SHAP (SHapley Additive exPlanations) to understand which features and time steps
are most important for the model's predictions.

In [ ]:
%pip install shap

import shap
import numpy as np
import tensorflow as tf

# Select background set for SHAP analysis
background = X_train[np.random.choice(len(X_train), 100, replace=False)]

def model_predict(x_flat):
    """Wrapper function for SHAP that handles flattened inputs."""
    x = x_flat.reshape(-1, X_train.shape[1], X_train.shape[2])
    return model.predict(x)

# Flatten background data
bg_flat = background.reshape(-1, X_train.shape[1] * X_train.shape[2])

# Create SHAP explainer
explainer = shap.KernelExplainer(model_predict, bg_flat)

# Analyze a single test sample
sample_idx = 0
x_to_explain = X_test[sample_idx:sample_idx + 1]
x_flat = x_to_explain.reshape(1, -1)

print("Computing SHAP values for sample analysis...")
shap_values = explainer.shap_values(x_flat, nsamples=200)

# Reshape SHAP values back to (time_steps, features)
sv = shap_values[0].reshape(X_train.shape[1], X_train.shape[2])

# Compute importance scores
feature_importance = np.sum(sv, axis=0)  # Sum over time steps
time_importance = np.sum(sv, axis=1)     # Sum over features

print(f"\nFeature importance shape: {feature_importance.shape}")
print(f"Time step importance shape: {time_importance.shape}")
print(f"Most important time step: {np.argmax(np.abs(time_importance))}")
print(f"Most important feature: {np.argmax(np.abs(feature_importance))}")

## SHAP Visualization and Analysis

Generate visualizations to understand model behavior across all test samples.

In [ ]:
import matplotlib.pyplot as plt

# Compute SHAP values for all test samples
N, T, F = X_test.shape
X_flat = X_test.reshape(N, T * F)

print("Computing SHAP values for all test samples...")
shap_values_all = explainer.shap_values(X_flat, nsamples=200)

# Average SHAP values across all test samples
mean_shap = np.mean(shap_values_all, axis=0).reshape(T, F)

# Create feature names
feature_names = [f"feat_{i}" for i in range(F)]
time_steps = np.arange(T)

# Plot heatmap of mean SHAP values
plt.figure(figsize=(12, 6))
plt.imshow(mean_shap, aspect='auto', cmap='RdBu_r', extent=[0, F, T, 0])
plt.colorbar(label="Mean SHAP Value")
plt.xlabel("Feature Index")
plt.ylabel("Time Step")
plt.title("Average SHAP Contributions Across Time and Features")
plt.yticks(time_steps)
plt.tight_layout()
plt.show()

print(f"Mean SHAP values shape: {mean_shap.shape}")

# Save results to CSV
mean_shap_df = pd.DataFrame(mean_shap, columns=feature_names)
mean_shap_df.to_csv('mean_shap_values.csv', index=False)
print("SHAP values saved to 'mean_shap_values.csv'")

## Temporal SHAP Analysis by Class

Analyze how different time steps contribute to predictions for each shot class.

In [ ]:
# Compute per-class temporal contributions
num_classes = len(le.classes_)
time_contrib = np.zeros((num_classes, T))

# Process SHAP values for each class
for c in range(num_classes):
    if isinstance(shap_values_all, list) and len(shap_values_all) > c:
        # Multi-class SHAP output
        sv_c = shap_values_all[c].reshape(-1, T, F)
    else:
        # Single output case - use overall SHAP values
        sv_c = shap_values_all.reshape(-1, T, F)
    
    # Sum over features to get time-step contributions
    time_per_sample = sv_c.sum(axis=2)
    # Average across all samples
    time_contrib[c] = time_per_sample.mean(axis=0)

# Plot temporal profiles for each class
fig, axes = plt.subplots(2, 4, figsize=(16, 8))
axes = axes.flatten()

for c in range(min(num_classes, 8)):  # Plot up to 8 classes
    axes[c].plot(range(T), time_contrib[c], marker='o', linewidth=2)
    axes[c].set_title(f"Temporal SHAP - {le.classes_[c]}")
    axes[c].set_xlabel("Time Step")
    axes[c].set_ylabel("SHAP Contribution")
    axes[c].grid(True, alpha=0.3)
    axes[c].axhline(y=0, color='black', linestyle='-', alpha=0.3)

# Hide unused subplots
for c in range(num_classes, len(axes)):
    axes[c].set_visible(False)

plt.tight_layout()
plt.show()

# Print summary statistics
print("\nTemporal Analysis Summary:")
for c in range(num_classes):
    peak_time = np.argmax(np.abs(time_contrib[c]))
    peak_value = time_contrib[c][peak_time]
    print(f"{le.classes_[c]}: Peak at time step {peak_time} (value: {peak_value:.4f})")

## Summary

This notebook demonstrates a complete pipeline for badminton shot classification using:

1. **MDS Embeddings**: Transform positional codes into continuous representations
2. **LSTM Architecture**: Capture temporal patterns in shot sequences
3. **Hyperparameter Tuning**: Optimize model performance automatically
4. **SHAP Analysis**: Understand model decisions and feature importance

The model achieves competitive accuracy on shot classification while providing
interpretable insights into which temporal patterns and features drive predictions.